In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/Usc/EE641/Project/")
print("current working directory is ", os.getcwd())

In [ ]:
!pip install torchio
!pip install yacs

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchio as tio
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import seg_models as sm
import utils
from utils import HDF5Dataset, training_loop, test_loop

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available()
                  else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [3]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomRotation((0, 5))])

test_transforms = transforms.Compose([transforms.ToTensor()])

In [4]:
infect_path = "HDF5SmallData/small_infect_data.hdf5"
infect_train_dataset = HDF5Dataset(infect_path, 'train', transform=train_transforms)
infect_val_dataset = HDF5Dataset(infect_path, 'val', transform=test_transforms)
infect_test_dataset = HDF5Dataset(infect_path, 'test', transform=test_transforms)

In [5]:
n_epochs = 2
batch_size = 2
stop = 5
num_works = 2

infect_Unet_micronet_m0 = sm.Unet(
    encoder_name="micronet_m0",       
    in_channels=1,                  
    classes=2,                      
).to(device)
infect_optimizer = optim.Adam(infect_Unet_micronet_m0.parameters(), lr=0.0001)
loss_fcn = nn.CrossEntropyLoss()

In [6]:
infect_train_loader = DataLoader(infect_train_dataset, batch_size=batch_size, 
shuffle=True, num_workers=num_works)
infect_val_loader = DataLoader(infect_val_dataset, batch_size=batch_size, 
shuffle=True, num_workers=num_works)

In [7]:
training_loop(
    infect_Unet_micronet_m0, infect_optimizer,
    loss_fcn, infect_train_loader,
    infect_val_loader, mask_name='infection mask',
    n_epochs=n_epochs, device=device, 
    save_path='trained_models/infect_Unet_micronet_m0.pt',
    stop = stop)

Epoch 1 Training Start:
Epoch 1, Batch 0
Epoch 1, Batch 10
Epoch 1, train loss: 0.376
Duration: 0.381 minutes
Epoch 1 Validation Start:
Epoch 1, Batch 0
Epoch 1, val loss: 0.180
Detect Improvement, Save Model
Epoch 2 Training Start:
Epoch 2, Batch 0
Epoch 2, Batch 10
Epoch 2, train loss: 0.317
Duration: 0.380 minutes
Epoch 2 Validation Start:
Epoch 2, Batch 0
Epoch 2, val loss: 0.176
Detect Improvement, Save Model


In [7]:
infect_test_loader = DataLoader(infect_test_dataset, batch_size=1, num_workers=num_works)

In [20]:
infect_test_loader.dataset[0][2]

0

In [8]:
infect_Unet_micronet_m0 = torch.load('trained_models/infect_Unet_micronet_m0.pt')
test_loop(infect_Unet_micronet_m0, infect_test_loader, 
mask_name='infection mask', device=device)

patient 0
accuracy mean: 3953239.250, accuracy standard deviation: 9683419.000
IoU mean: 0.000, IoU standard deviation: 0.000
DSC mean: 211626.797, DSC standard deviation: 518377.656


(3953239.25,
 9683419.0,
 1.1871539617386587e-31,
 2.9079214681432633e-31,
 211626.796875,
 518377.65625)

In [6]:
lung_path = "HDF5Data/lung_data.hdf5"
lung_train_dataset = HDF5Dataset(lung_path, 'train', transform=train_transforms)
lung_val_dataset = HDF5Dataset(lung_path, 'val', transform=test_transforms)
lung_test_dataset = HDF5Dataset(lung_path, 'test', transform=test_transforms)